### Konstantina Andronikou 

## This Notebook implements Predicate Ellipsis 

In [21]:
from allennlp_models.pretrained import load_predictor

In [22]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [23]:
from checklist.pred_wrapper import PredictorWrapper

In [24]:
#removing warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("John saw three dogs and Mary two.")
output

{'verbs': [{'verb': 'saw',
   'description': '[ARG0: John] [V: saw] [ARG1: three dogs and Mary two] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['John', 'saw', 'three', 'dogs', 'and', 'Mary', 'two', '.']}

In [25]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [26]:
d = ["John saw three dogs and Mary two"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'saw',
     'description': '[ARG0: John] [V: saw] [ARG1: three dogs and Mary two]',
     'tags': ['B-ARG0',
      'B-V',
      'B-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1',
      'I-ARG1']}],
   'words': ['John', 'saw', 'three', 'dogs', 'and', 'Mary', 'two']}],
 array([1.]))

In [27]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [28]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [29]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [30]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name1']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [31]:
editor = Editor()

# create examples
t = editor.template('{first_name} saw three dogs and {first_name1} two.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Joe', 'first_name1': 'Ian'}, {'first_name': 'Julia', 'first_name1': 'Peter'}, {'first_name': 'Robert', 'first_name1': 'Diane'}, {'first_name': 'Kenneth', 'first_name1': 'Peter'}, {'first_name': 'Ruth', 'first_name1': 'Barbara'}, {'first_name': 'Roy', 'first_name1': 'Annie'}, {'first_name': 'Paul', 'first_name1': 'Leslie'}, {'first_name': 'Philip', 'first_name1': 'Kim'}, {'first_name': 'Ian', 'first_name1': 'Emma'}, {'first_name': 'Matthew', 'first_name1': 'Maria'}, {'first_name': 'Donald', 'first_name1': 'Frank'}, {'first_name': 'Margaret', 'first_name1': 'Dick'}, {'first_name': 'John', 'first_name1': 'Jimmy'}, {'first_name': 'Simon', 'first_name1': 'Edward'}, {'first_name': 'Fred', 'first_name1': 'Dorothy'}, {'first_name': 'Sally', 'first_name1': 'Alice'}, {'first_name': 'Richard', 'first_name1': 'Heather'}, {'first_name': 'Evelyn', 'first_name1': 'Claire'}, {'first_name': 'Robert', 'first_name1': 'Patricia'}, {'first_name': 'Pamela', 'first_name1': 'Tim'}, {'fir

In [32]:
with open('dataset/Predicate_Ellipsis.txt', 'w') as f:
    print(t.data, file = f)

In [33]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Predicate_Ellipsis.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 994 examples
Test cases:      994
Fails (rate):    994 (100.0%)

Example fails:
[ARG0: Michelle] [V: saw] [ARG1: three dogs and Chris two] .
----
[ARG0: Lucy] [V: saw] [ARG1: three dogs and Hugh two] .
----
[ARG0: Frank] [V: saw] [ARG1: three dogs and John two] .
----
['Joe', 'saw', 'three', 'dogs', 'and', 'Ian', 'two', '.'] [False]
['Julia', 'saw', 'three', 'dogs', 'and', 'Peter', 'two', '.'] [False]
['Robert', 'saw', 'three', 'dogs', 'and', 'Diane', 'two', '.'] [False]
['Kenneth', 'saw', 'three', 'dogs', 'and', 'Peter', 'two', '.'] [False]
['Ruth', 'saw', 'three', 'dogs', 'and', 'Barbara', 'two', '.'] [False]
['Roy', 'saw', 'three', 'dogs', 'and', 'Annie', 'two', '.'] [False]
['Paul', 'saw', 'three', 'dogs', 'and', 'Leslie', 'two', '.'] [False]
['Philip', 'saw', 'three', 'dogs', 'and', 'Kim', 'two', '.'] [False]
['Ian', 'saw', 'three', 'dogs', 'and', 'Emma', 'two', '.'] [False]
['Matthew', 'saw', 'three', 'dogs', 'and', 'Maria', 'two', '.'] [False]
['Donald', 'saw', 'three

In [34]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Predicate_Ellipsis.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 